In [1]:
import nbimporter
import csv
import climate
import sun_position

Importing Jupyter notebook from climate.ipynb
Importing Jupyter notebook from sun_position.ipynb


# 日よけ効果係数算定ツール Modules

- 一次目標：日よけ効果係数算定ツールのpython上での再現


## D. 地点と日射量

### D.3 正時±30分で太陽が地平線上にある時間刻み数のカウント (仕様書6.2 式(3)及び図5中の$n_H$の計算)

- 算定ツール標準の時間分割数$n_{\Delta t}$は、$6$ ($10$分刻み)
  - $1$時間を$1$分割もしくは$2$以上の偶数で分割する
  
  
- 正時$\pm 30$分間で太陽が地平線上にある(太陽高度$>0$)時間刻み数をカウントして、$n_H$を計算

In [2]:
""" 式(3),図5中のNhの計算 """
import sys

def calc_Nh(Latitude, Longitude, NDay, NHour, NDT):

    deltad = sun_position.calc_deltad(NDay)
    eed = sun_position.calc_eed(NDay)
   
    Nh = 0
    if NDT == 1:
        if sun_position.calc_sinh(Latitude, deltad, sun_position.calc_Tdt(Longitude, eed, NHour)) > 0:
            Nh += 1        
    elif NDT > 0 and NDT % 2 == 0:
        # 1/1 0時, 各日24時において、赤緯と均時差にズレが生じるが、
        # 白夜でなければ日が昇らないので実質影響なし → 放置
        sinh0 = [sun_position.calc_sinh(Latitude, deltad, sun_position.calc_Tdt(Longitude, eed, NHour + m / NDT)) 
                                                  for m in range(-int(NDT/2),int(NDT/2)+1)]
        Nh = ( sum(x > 0 for x in sinh0) 
             - (0.5 if sinh0[0] > 0 else 0) - (0.5 if sinh0[int(NDT)] > 0 else 0) )
        
        #以下のコメントアウト部分を上の式二つで置き換え    
        #if calc_sinh(Latitude, deltad, calc_Tdt(Longitude, eed, NHour - 0.5)) > 0: #正時の30分前        
        #    Nh += 0.5
        #if calc_sinh(Latitude, deltad, calc_Tdt(Longitude, eed, NHour + 0.5)) > 0: #正時の30分後
        #    Nh += 0.5
        #for m in range(int(NDT/2+1), int(NDT)):
        #    if calc_sinh(Latitude, deltad, calc_Tdt(Longitude, eed, NHour -1 + m / NDT)) > 0:
        #        Nh += 1
        #for m in range(0, int(NDT/2)):
        #    if calc_sinh(Latitude, deltad, calc_Tdt(Longitude, eed, NHour + m / NDT)) > 0:
        #        Nh += 1            
    else:
        sys.exit("1時間あたりの時間分割数は1もしくは2以上の偶数とする必要があります")

    return Nh

### D.4 $1/n_{\Delta t}$時間間隔での日射量 (仕様書6.2 式(3)の計算, 図5参照)

- 算定ツール標準の時間分割数$n_{\Delta t}$は、$6$ ($10$分刻み)
- 正時$\pm 30$分間で太陽が地平線上にある(太陽高度$>0$)時間刻み数をカウントして、$n_H$を計算
- 法線面直達日射量, 水平面天空日射量をそれぞれに適用し、時間刻みにおける日射量を算定する

In [3]:
""" 式(3)の S'HM の計算 """
def calc_Sdhm(MM, NDT, sinh, Sh, Shp, Nh, Nhp):
    # MM:1時間の内の(1/NDT)間隔の順番, 正時がMM=0, MM=0～NDT-1
    # NDT = 6                    # 1時間の分割数,ツールの標準は6分割
    # sinh:hsdtの正弦
    # Sh: 法線面直達日射量[kcal/(m2h)]
    # Shp: 法線面直達日射量[kcal/(m2h)] 1時間後
    # Nh: 太陽が地平線上にある時間（分割した後の⊿tの数）
    # Nhp: 太陽が地平線上にある時間（分割した後の⊿tの数） 1時間後
    
    Sdhm = 0
    if sinh > 0:
        Sdhm = ( ( (Sh  / Nh  if MM <= NDT/2 and Nh  > 0 else 0)
                 + (Shp / Nhp if MM >= NDT/2 and Nhp > 0 else 0) )
               / (2 if MM == NDT/2 else 1) )
        #以下のコメントアウト部分を上式で置き換え
        #if MM < NDT/2 and Nh > 0:
        #    Sdhm = Sh / Nh
        #elif MM == NDT/2:
        #    if Nh > 0:
        #        Sdhm += Sh / Nh / 2
        #    if Nhp > 0:
        #        Sdhm += Shp / Nhp / 2            
        #elif NDT/2 < MM and Nhp > 0:
        #    Sdhm = Shp / Nhp
    else:
        Sdhm = 0
        
    return Sdhm  

### D.5 窓ガラスの入射角特性読み込み

- `\IncidentAngleCharacteristics.csv`
  - $1$行目はヘッダ："入射角特性", $\eta_{max}$, $\eta_{isr}$, $k_0$～$k_7$
  - $2$行目以下に入射角特性のデータを記入
    - $1$列：$ID$
    - $2$列：直達日射に対する入射角特性最大値(入射角$0$) $\eta_{max}$
    - $3$列：天空・反射日射に対する入射角特性(遮蔽なしの場合) $\eta_{isr}$
    - $4～11$列：$\eta_{j,d,t}(\theta_{j,d,t})$ 算定式の係数$k_n$$(n=0～7)$
       $$\eta_{j,d,t}(\theta_{j,d,t}) = \sum_{n=0}^7 k_n \cos^n \theta_{j,d,t}$$
  - デフォルトとして、以下を設定
    - $ID=0$：日よけ効果係数内で入射角特性非考慮 → $\eta_{j,d,t}(\theta_{j,d,t}) = 1$
    - $ID=1$：解説書の入射角特性(「平成25年度省エネルギー基準に準拠した算定・判断の方法及び解説 I 非住宅建築物 第二版(連合印刷センター, 平成26年○月○日)」, pp.168-170, 式(2.1.25),(2.1.28),(2.1.32))
       $$\eta_{j,d,t}(\theta_{j,d,t}) = 2.3920 \cos \theta_{j,d,t} -3.8636 \cos^3 \theta_{j,d,t} + 3.7568 \cos^5 \theta_{j,d,t} - 1.3952 \cos^7 \theta_{j,d,t} $$
    - 他の特性を入れる際には、$ID$を違えて、`\IncidentAngleCharacteristics.csv` に追加する。

In [4]:
""" 入射角特性データセットを \IncidentAngleCharacteristics.csv から読み込む  """
# \地域区分+日射量データ+窓面入射角特性.xlsx "入射角特性"シート
#   → \IncidentAngleCharacteristics.csv を作成 → 読み込み
import pandas as pd
import sys

def input_IncidentAngleCharacteristics(ID, Path00, FileName00):
    # Path00 = "./SCFConfig01/"
    # FileName00 = "IncidentAngleCharacteristics.csv"

    csv_input = pd.read_csv(filepath_or_buffer=Path00+FileName00, encoding="ms932", sep=",")
    if csv_input.columns[0]!="入射角特性":
        sys.exit("ファイル内に貼り付けたテスト条件が違います")

    ID0="none"    
    for i in range(len(csv_input)):
        if ID == csv_input.values[i][0]:
            [ID0, etamax, etaisr, etakk] = [csv_input.values[i][0],csv_input.values[i][1]
                                            ,csv_input.values[i][2],csv_input.values[i][3:11]]
    if ID0=="none":
        sys.exit("指定したIDの入射角特性がありません")            
    
    return [ID0, etamax, etaisr, etakk]

### D.6 直達日射に対する窓ガラスの入射角特性 (緑本非住宅第二版pp.169 式(2.1.28)準拠)

- 「平成25年度省エネルギー基準に準拠した算定・判断の方法及び解説 I 非住宅建築物 第二版(連合印刷センター, 平成26年○月○日)」pp.168-170参照
- 入射角$\theta_{j,d,t}[deg]$, 太陽高度$h_{S,d,t}[deg]$, 太陽方位角$A_{ZS,d,t}[deg]$, 外壁$j$の方位角$A_{ZW,i}[deg]$
- 入射角特性算定式の係数$k_n$$(n=0～7)$, 日付$d$時刻$t$における入射角特性$\eta_{j,d,t}$

$$\cos \theta_{j,d,t} = \cos h_{S,d,t} \cos (A_{ZS,d,t} - A_{ZW,i}) \qquad (2.1.26) $$
$$\eta_{j,d,t}(\theta_{j,d,t}) = \sum_{n=0}^7 k_n \cos^n \theta_{j,d,t} \qquad (2.1.28') $$

In [5]:
""" 式(2.1.26),(2.1.28') """
import numpy as np

def calc_costheta(Azwjdt, cosh):

    costheta = max(cosh * np.cos(np.radians(Azwjdt)),0)

    return costheta


def calc_etajdt(costheta, etakk):

    etajdt = sum([etakk[i]*costheta**i for i in range(len(etakk))])
        
    return etajdt

## E. 日よけ効果係数

### E.1 日よけ効果係数の算出 (仕様書6.1 式(2)準拠)

- $\gamma_{dsr,j,x+}$：窓等$j$に対して太陽が$x+$側に位置する際の直達日射に対する日よけ効果係数(式$(13)$)
- $\gamma_{dsr,j,x-}$：窓等$j$に対して太陽が$x-$側に位置する際の直達日射に対する日よけ効果係数(式$(17)$)
- $\gamma_{isr,j,y+}$：天空日射に対する日よけ効果係数(式$(21)$) → C.3参照
- $\gamma_{isr,j,y-}$：反射日射に対する日よけ効果係数(式$(24)$) → C.5参照
- $\eta_{j,d,t}$：日付$d$、時刻$t$における窓等$j$の入射角特性 → D.6参照
- $S_{D,d,t}$：日付$d$、時刻$t$における法線面直達日射量$[kcal/m^2]$
- $S_{S,d,t}$：日付$d$、時刻$t$における水平面天空日射量$[kcal/m^2]$
- $\eta_{isr}$：天空・反射日射に対する入射角特性(遮蔽なしの場合)$[-]$
  - 緑本、仕様書では$\;\eta_{isr} = 0.808\;$
  
  
- 式(2)のうち、定数$0.5$は垂直面からみた天空・地表面の形態係数、$0.1$は地表面における日射反射率である。

$$ \begin{align}
\gamma_{wind,j} 
= \big[&\hspace{4pt}\gamma_{dsr,j,x+} \times \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} 
        (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
  + &\hspace{4pt} \gamma_{dsr,j,x-} \times \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} 
        (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
  + &\hspace{4pt}\gamma_{isr,j,y+} \times \sum (\eta_{isr} \times 0.5 \times S_{S,d,t} \\
  + &\hspace{4pt}\gamma_{isr,j,y-} \times \sum \{\eta_{isr} \times 0.1 \times 0.5 
        \times (S_{S,d,t} + S_{D,d,t} \sin h_{s,d,t}) \} \big] \\
\big/ \hspace{4pt} \big[&\sum_{-90<A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \\
    + &\hspace{4pt}\sum \{\eta_{isr} \times 0.5 \times S_{S,d,t} 
        +  \eta_{isr} \times 0.1 \times 0.5 \times (S_{S,d,t} + S_{D,d,t} \sin h_{s,d,t}) \} \big] \qquad (2)
\\
\end{align} $$

- $A_{wind,j}$：窓等$j$の面積($=x_2y_2$)$[m^2]$
- $A_{wind,j,x+,d,t}$：太陽が$x+$側に位置する日付$d$、時刻$t$において窓等$j$の直達日射が当たる部分の面積$[m^2]$
- $A_{wind,j,x-,d,t}$：太陽が$x-$側に位置する日付$d$、時刻$t$において窓等$j$の直達日射が当たる部分の面積$[m^2]$

$$ \begin{align}
\gamma_{dsr,j,x+} 
= \big\{ & \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} (A_{wind,j,x+,d,t}\hspace{2pt}  
        S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \big\} \\
\big/ \big\{ &\hspace{2pt} A_{wind,j} \sum_{-90<A_{ZW,j,d,t}<0, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t})  \} \big\} \qquad (13)
\\
\end{align} $$

$$ \begin{align}
\gamma_{dsr,j,x-} 
= \big\{ & \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (A_{wind,j,x-,d,t} \hspace{2pt} 
        S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t}) \big\} \\
\big/ \big\{ &\hspace{2pt} A_{wind,j} \sum_{0 \leq A_{ZW,j,d,t}<90, \; h_{s,d,t}>0} (S_{D,d,t} \; \eta_{j,d,t} \cos h_{s,d,t} \cos A_{ZW,j,d,t})  \} \big\} \qquad (17)
\\
\end{align} $$



### E.2 式(2)分子分母中の時刻$t$における各成分の計算

In [6]:
""" 直達成分の分子分母 """
def calc_dendsr00(Sddhm, etajdt, costheta, Awj): #(2)式分母中直達成分×窓面積
    
    dendsr00 = Awj * Sddhm * etajdt * costheta

    return dendsr00


def calc_numdsr00(Sddhm, etajdt, costheta, Ax): #(2)式分子中直達成分×窓面積←(13)(17)式

    numdsr00 = Ax * Sddhm * etajdt * costheta
    
    return numdsr00


""" 天空成分の分子分母 """
def calc_denisryp00(Ssdhm, etaisr, Awj): #(2)式分母中天空成分×窓面積
    
    denisryp00 = Awj * etaisr * 0.5 * Ssdhm
    
    return denisryp00


def calc_numisryp00(Ssdhm, etaisr, Awj, gammayp): #(2)式分子中天空成分×窓面積

    numisryp00 = gammayp * Awj * etaisr * 0.5 * Ssdhm
    # numisryp00 = gammayp * denisryp00
    
    return numisryp00    


""" 反射成分の分子分母 """
def calc_denisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj): #(2)式分母中反射成分×窓面積
    
    denisrym00 = Awj * etaisr * 0.1 * 0.5 * (Ssdhm + Sddhm * sinh)
    
    return denisrym00


def calc_numisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj, gammaym): #(2)式分子中反射成分×窓面積

    numisrym00 = gammaym * Awj * etaisr * 0.1 * 0.5 * (Ssdhm + Sddhm * sinh)
    # numisrym00 = gammaym * denisrym00
    
    return numisrym00


""" 分子分母それぞれの和 """
def calc_SCF00(Sddhm, Ssdhm, etajdt, etaisr, costheta, sinh, Awj, Ax, gammayp, gammaym):
    
    SCF00 = [0,0]
    
    # SCF00[0]:(2)式の分子への加算分(窓面積をかけた値として)
    SCF00[0] = ( calc_numdsr00(Sddhm, etajdt, costheta, Ax)
               + calc_numisryp00(Ssdhm, etaisr, Awj, gammayp)
               + calc_numisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj, gammaym) )
    # SCF00[1]:(2)式の分母への加算分(窓面積をかけた値として)
    SCF00[1] = ( calc_dendsr00(Sddhm, etajdt, costheta, Awj)             
               + calc_denisryp00(Ssdhm, etaisr, Awj)
               + calc_denisrym00(Sddhm, Ssdhm, etaisr, sinh, Awj) )

    return SCF00


### E.3 式(2)分子分母の期間積算処理

- 式(2)中の時刻$t$における分子および分母を期間積算する。
  - 暖房期、冷房期、非空調期
  - 各月毎
  - 各月、各時刻毎
    - 時刻毎は正時$\pm30$分としたいが、本ツール内でのデータの取り扱いからずれる。前時刻の`MM` $=$ `NDT`$/2～$同時刻の`MM` $=$ `NDT`$/2-1$の間を正時の積算値として計上する。
    

- `SCF[h][i][j]`に格納して積算
  - `h`$=0$：分子, `h`$=1$：分母, `h`$=2$：効果係数
  - `i`$=-2$：冷房期積算, `i`$=-1$：暖房期積算, `i`$=0$：非空調期積算, `i`$=1～12$：各月で積算
  - `j`$=-1$：日積算, `j`$=0～24$：各時刻で積算  

  



In [7]:
""" 期間積算処理 → 日よけ効果係数算出 """

def Output_ShadingCorrectionFactor(SRHour, NDT, SCF01):

    SCF = [[[0 for j in range(-1,25)] for i in range(-2,13)] for h in range(0,3)] 
        # SCF[h][i][j]に格納して積算
        # h = 0：分子, h = 1：分母, h = 2：効果係数
        # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
        #         i = 1～12：各月で積算
        # j = -1：日積算, j = 0～24：各時刻で積算  
        
    """ +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 1時間のループ ++++ """       
    for Hour00 in range(8760):      # ← 12/31 24時は回さないのでHour00=8759がループの最後
        
        """ 日時の計算(A.1) """
        [NDay,NHour] = sun_position.calc_NDayNHour(Hour00)           
        
        """ 「月」の計算(D.2) """
        Month = climate.calc_Month(SRHour[Hour00][0])        
 
        """ +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ (1/NDT)時間のループ ++++ """ 
            # MM:1時間の内の(1/NDT)間隔の順番, 正時がMM=0, MM=0～NDT-1
        for MM in range(NDT):
            """ 時刻TTの計算(A.1) """
                # TT:時間分割MM毎の時刻[hour]
            TT = sun_position.calc_TT(NHour, NDT, MM)
            
            """ 時刻Hour01の計算(A.1) """
                # Hour01:前時刻のMM=NDT/2～同時刻のMM=NDT/2-1の時間分割を、その正時に属するものとして
                #        扱うための時刻 → 効果係数算定のための積算用
            Hour01 = sun_position.calc_Hour01(TT)
            
            """ 暖冷房期間のタグ """            
            HCTag = -SRHour[Hour00][3]

            """ 分子分母の期間,月,時間毎の積算(E.3) """
            for h in range(0, 2):
                SCF[h][HCTag][Hour01] += SCF01[h][Hour00][MM]
                SCF[h][HCTag][-1]     += SCF01[h][Hour00][MM]        
                SCF[h][Month][Hour01] += SCF01[h][Hour00][MM]
                SCF[h][Month][-1]     += SCF01[h][Hour00][MM]                      
                    # SCF[h][i][j]に格納して積算
                    # h = 0：分子, h = 1：分母, h = 2：効果係数
                    # i = -2：冷房期積算, i = -1：暖房期積算, i = 0：非空調期積算,
                    #         i = 1～12：各月で積算
                    # j = -1：日積算, j = 0～24：各時刻で積算             
                    
    """ 期間,月,時間毎の日よけ効果係数算出(E.1) """
    for i in range(-2,13):
        for j in range(-1,25):
            if SCF[1][i][j] != 0:
                SCF[2][i][j] = SCF[0][i][j] / SCF[1][i][j]
            else:
                SCF[2][i][j] = 0

    return SCF

